<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Terminate EC2 Instances Without Valid Lifetime Tag using unSkript legos.</b>
</div>

<br>

<center><h2>Terminate EC2 Instances Without Valid Lifetime Tag</h2></center>

# Steps Overview
  - When an Auto Scaling group launches instances, it adds tags to the instances during              resource creation rather than after the resource is created.

  - The Auto Scaling group automatically adds a tag to instances with a key of                       aws:autoscaling:groupName and a value of the Auto Scaling group name.

  - If you specify instance tags in your launch template and you opted to propagate your             group's tags to its instances, all the tags are merged. If the same tag key is specified         for a tag in your launch template and a tag in your Auto Scaling group, then the tag value       from the group takes precedence.

  - When you attach existing instances, the Auto Scaling group adds the tags to the instances,       overwriting any existing tags with the same tag key. It also adds a tag with a key of           aws:autoscaling:groupName and a value of the Auto Scaling group name.

  - When you detach an instance from an Auto Scaling group, it removes only the                     aws:autoscaling:groupName tag.

Here we will use unSkript Filter AWS EC2 Instances Without Lifetime Tag Lego. This lego takes lifetime_tag and region as input. This input is used to get all the EC2 instances which don't have lifetime tag

In [6]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_filter_ec2_without_lifetime_tag_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})


@beartype
def aws_filter_ec2_without_lifetime_tag(handle, lifetime_tag: str, region: str) -> List:
    """aws_filter_ec2_without_lifetime_tag Returns an array of instances matching tags.

        :type handle: object
        :param handle: Object returned from task.validate(...)..

        :type lifetime_tag: string
        :param lifetime_tag: Tag to filter Instances.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :rtype: Array of instances which not having lifetime tag.
    """
    # Input param validation.

    ec2Client = handle.client('ec2', region_name=region)
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")

    result = []
    for reservation in res:
        for instance in reservation['Instances']:
            try:
                tagged_instance = instance['Tags']
                tag_keys = [tags['Key'] for tags in tagged_instance]
                if lifetime_tag not in tag_keys:
                    result.append(instance['InstanceId'])
            except Exception as e:
                result.append(instance['InstanceId'])

    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "lifetime_tag": "Lifetime_Tag",
    "region": "Region"
    }''')
task.configure(outputName="InstanceId")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_ec2_without_lifetime_tag, lego_printer=aws_filter_ec2_without_lifetime_tag_printer, hdl=hdl, args=args)

Here we will use unSkript Terminate AWS Instances Lego. This lego takes instance_id and region as input. This input is used to terminate EC2 instances which don't have lifetime tag

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_terminate_instance_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_terminate_instance(handle, instance_ids: List, region: str) -> Dict:
    ec2Client = handle.client('ec2', region_name=region)
    res = ec2Client.terminate_instances(InstanceIds=instance_ids)

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "instance_ids": "InstanceId",
    "region": "Region"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_terminate_instance, lego_printer=aws_terminate_instance_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to perform AWS action of collecting (and then terminating) any EC2 instances which don't have a lifetime tag. To view the full platform capabilities of unSkript please visit https://unskript.com